In [1]:
import time
import os

import pandas as pd
from pycoingecko import CoinGeckoAPI

RD = '/home/team_crypto/2022_crypto_return/02_rdata/'
WD = '/home/team_crypto/2022_crypto_return/03_wdata/1100_coingecko/'

cg = CoinGeckoAPI()
cg.ping()

SSLError: HTTPSConnectionPool(host='api.coingecko.com', port=443): Max retries exceeded with url: /api/v3/ping (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [ ]:
def to_unix_time(x):
    return int(pd.to_datetime(x).timestamp())

def to_timestamp(x):
    return pd.to_datetime(x, unit='ms', origin='unix')

CoinGecko API文档：https://www.coingecko.com/en/api/documentation

pycoingecko主页：https://github.com/man-c/pycoingecko

# Full List

In [ ]:
coins = pd.read_excel(f'{RD}CoinGecko Token API List.xlsx')
coins.shape
coins.sample(3)

In [ ]:
coins.loc[coins['Id'] == 'litecoin']

In [ ]:
# 用来获取全部数据过程中记录完成情况
# coins['attempt'] = 0
# coins.to_csv(f'{WD}coin_list.csv', index=False)

# Test

In [ ]:
# s1 = 'litecoin'
s1, s2, s3, *_ = coins.loc[6542, :].values
ft, tt = to_unix_time('2019-11-01'), to_unix_time('2020-03-31')

s1
s2, s3
ft, tt

In [ ]:
ret = cg.get_coin_market_chart_range_by_id(
    id=s1,
    vs_currency='usd', 
    from_timestamp=ft, 
    to_timestamp=tt
)

len(ret['prices'])
if len(ret['prices']) == 0:
    print(idx, 'No return.')

In [ ]:
# type(ret)
# ret

In [ ]:
dfs = []

for var in ret.keys():
    df = pd.DataFrame(ret[var])
    df.columns = ['time', var]
    df['time'] = df['time'].map(to_timestamp)
    df.set_index('time', inplace=True)
    dfs.append(df)
    
dfm = pd.concat(dfs, axis=1)
dfm.shape
dfm.head()
dfm.tail()

In [ ]:
dfm['id'] = s1
dfm['symbol'] = s2
dfm['name'] = s3
# dfm['source'] = 'CoinGecko'
dfm.sample()

In [ ]:
dfm.reset_index().to_parquet(f'{WD}price/{s1}.pq')

In [ ]:
df = pd.read_parquet(f'{WD}price/{s1}.pq')
df.sample(3)

In [ ]:
df.set_index('time', inplace=True)
df['prices'].plot()

# Run

In [ ]:
coins = pd.read_csv(f'{WD}coin_list.csv')
coins.shape
coins['attempt'].value_counts()
coins.sample(3)

In [ ]:
done = set([f.replace('.pq', '') for f in os.listdir(f'{WD}price') if 'pq' in f])
coins.loc[coins['Id'].isin(done), 'attempt'] = -1

todo = set(coins.loc[coins['attempt'].between(0, 10), 'Id'])

len(todo)

In [ ]:
todo = ['ethereum', 'ethereum-classic']  # , 'ripple', 'litecoin', 'tether'

In [ ]:
ft, tt = to_unix_time('2019-11-01'), to_unix_time('2020-03-31')

for idx in todo:
    time.sleep(5)  # 免费API限制每分钟爬50次，最小间隔为1.2秒，安全起见此处设为5秒
    try: 
        s1, s2, s3, _ = coins.loc[coins['Id'] == idx, :].values[0]
        print(f"正在获取{s1}……")
        
        ret = cg.get_coin_market_chart_range_by_id(
            id=s1,
            vs_currency='usd', 
            from_timestamp=ft, 
            to_timestamp=tt
        )
        
        if len(ret['prices']) == 0:
            # print(idx, 'No return.')
            coins.loc[coins['Id'] == idx, 'attempt'] = -2  # -2表示虽然成功获取了数据，但历史上并无数据
            continue
        
        dfs = []

        for var in ret.keys():
            df = pd.DataFrame(ret[var])
            df.columns = ['time', var]
            df['time'] = df['time'].map(to_timestamp)
            df.set_index('time', inplace=True)
            dfs.append(df)

        dfm = pd.concat(dfs, axis=1)
        # dfm.shape

        dfm['id'] = s1
        dfm['symbol'] = s2
        dfm['name'] = s3

        dfm.reset_index().to_parquet(f'{WD}price/{s1}.pq')

        coins.loc[coins['Id'] == idx, 'attempt'] = -1
        
    except:
        val = coins.loc[coins['Id'] == idx, 'attempt'].values[0] + 1
        coins.loc[coins['Id'] == idx, 'attempt'] = val
        print(f"第{val}次获取{s1}失败……")
        # raise
        
coins.to_csv(f'{WD}coin_list.csv', index=False)

In [ ]:
coins.to_csv(f'{WD}coin_list.csv', index=False)

# Aggregate

In [ ]:
df_lst = []

for file in os.listdir(f'{WD}price'):
    if '.pq' in file:
        df = pd.read_parquet(f'{WD}price/{file}')
        df_lst.append(df)
        
dfm = pd.concat(df_lst)
dfm.shape
dfm.head(3)

In [ ]:
dfm['id'].value_counts()

In [ ]:
dfm.reset_index(drop=True).to_parquet(f'{WD}1101_coingecko_price.pq')